## scikit-learn sample_weight compliance report

This notebook runs compliance tests on all scikit-learn estimators. Estimator as inspected to check whether they are expected to have a stochastic fit or not. If the fit is stochastic, a dedicated statistical test is performed, otherwise a deterministic estimator check is run instead.

In [1]:
import os
os.environ["SCIPY_ARRAY_API"] = "1"

In [2]:
import sklearn

sklearn.show_versions()


System:
    python: 3.12.4 | packaged by conda-forge | (main, Jun 17 2024, 10:13:44) [Clang 16.0.6 ]
executable: /Users/shrutinath/micromamba/envs/scikit-learn/bin/python
   machine: macOS-14.3-arm64-arm-64bit

Python dependencies:
      sklearn: 1.7.dev0
          pip: 24.0
   setuptools: 75.8.0
        numpy: 2.0.0
        scipy: 1.14.0
       Cython: 3.0.10
       pandas: 2.2.2
   matplotlib: 3.9.0
       joblib: 1.4.2
threadpoolctl: 3.5.0

Built with OpenMP: True

threadpoolctl info:
       user_api: blas
   internal_api: openblas
    num_threads: 8
         prefix: libopenblas
       filepath: /Users/shrutinath/micromamba/envs/scikit-learn/lib/libopenblas.0.dylib
        version: 0.3.27
threading_layer: openmp
   architecture: VORTEX

       user_api: openmp
   internal_api: openmp
    num_threads: 8
         prefix: libomp
       filepath: /Users/shrutinath/micromamba/envs/scikit-learn/lib/libomp.dylib
        version: None


In [3]:
from inspect import signature
import traceback
import warnings
import pandas as pd
from sklearn.base import is_clusterer
from sklearn.utils import all_estimators
from sklearn.utils.estimator_checks import check_sample_weight_equivalence_on_dense_data
from sklearn.exceptions import ConvergenceWarning
import threadpoolctl

import sys
sys.path.insert(1,'../src/')
from sample_weight_audit import check_weighted_repeated_estimator_fit_equivalence
from sample_weight_audit.exceptions import UnexpectedDeterministicPredictions
from sample_weight_audit.sklearn_stochastic_params import STOCHASTIC_FIT_PARAMS

# HistGradientBoostingClassifier trashes the OpenMP thread pool on repeated
# small fits.
threadpoolctl.threadpool_limits(limits=1, user_api="openmp")
warnings.filterwarnings("ignore", category=RuntimeWarning)  # division by zero in AdaBoost
warnings.filterwarnings("ignore", category=ConvergenceWarning)  # liblinear can fail to converge
warnings.filterwarnings("ignore", category=UserWarning)  # KBinsDiscretizer with collapsed bins

In [4]:
from sklearn.linear_model import LogisticRegressionCV

ESTIMATORS_TO_SKIP = [
    LogisticRegressionCV,  # too slow and already somewhat tested by LogisticRegression
]

In [6]:
STAT_TEST_DIM = 30
N_STOCHASTIC_FITS = 100
N_STOCHASTIC_ESTIMATORS = 36  # measured a posteriori
# BONFERRONI_CORRECTION = 1 / (N_STOCHASTIC_ESTIMATORS * STAT_TEST_DIM)
BONFERRONI_CORRECTION = 1 / STAT_TEST_DIM

TEST_THRESHOLD = 0.05 * BONFERRONI_CORRECTION


statistical_test_results = []
deterministic_test_results = []
missing_sample_weight_support = []
errors = []


for est_name, est_class in all_estimators(
    type_filter=["classifier", "regressor", "cluster", "transformer"]
):
    if est_class in ESTIMATORS_TO_SKIP:
        print(f"Skipping {est_name}")
        continue

    if "sample_weight" not in signature(est_class.fit).parameters:
        print(f"⚠ {est_name} does not support sample_weight")
        missing_sample_weight_support.append(est_name)
        continue

    try:
        est = est_class(**STOCHASTIC_FIT_PARAMS.get(est_class, {}))
    except TypeError as e:
        print(f"⚠ {est_name} failed to instantiate: {e}")
        continue
    
    print(f"Evaluating {est}")
    try:
        result = check_weighted_repeated_estimator_fit_equivalence(
            est,
            test_name="kstest",
            n_stochastic_fits=N_STOCHASTIC_FITS,
            random_state=0,
        )
        pass_or_fail = "✅" if result.p_value > TEST_THRESHOLD else "❌"
        print(
            f"{pass_or_fail} {est_name}: (p_value: {result.p_value:.3f})"
        )
        statistical_test_results.append(result)
    except UnexpectedDeterministicPredictions:
        # The estimator parametrization led to deterministic behavior, which is
        # unexpected. Run the deterministic check to investigate instead.
        print(f"⚠ {est_name} with different random states led to the same predictions")
        try:
            check_sample_weight_equivalence_on_dense_data(
                est_name, est.set_params(random_state=0)
            )
            print(f"✅ {est} passed the deterministic check")
            deterministic_test_results.append((est, None))
        except Exception as e:
            print(f"❌ {est} failed the deterministic check")
            deterministic_test_results.append((est, e))
    except Exception as e:
        print(f"❌ {est} error with: {e}")
        errors.append((est, e))

results_df = pd.DataFrame([r.to_dict() for r in statistical_test_results])

⚠ ARDRegression does not support sample_weight
Evaluating AdaBoostClassifier(estimator=DecisionTreeClassifier(max_features=0.5,
                                                    min_weight_fraction_leaf=0.1))


100%|██████████| 100/100 [00:05<00:00, 17.07it/s]


✅ AdaBoostClassifier: (p_value: 0.815)
Evaluating AdaBoostRegressor(estimator=DecisionTreeRegressor(max_features=0.5,
                                                  min_weight_fraction_leaf=0.1))


100%|██████████| 100/100 [00:04<00:00, 21.16it/s]


✅ AdaBoostRegressor: (p_value: 0.078)
⚠ AdditiveChi2Sampler does not support sample_weight
⚠ AffinityPropagation does not support sample_weight
⚠ AgglomerativeClustering does not support sample_weight
Evaluating BaggingClassifier()


...

❌ SplineTransformer: (p_value: 0.000)
⚠ StackingClassifier failed to instantiate: StackingClassifier.__init__() missing 1 required positional argument: 'estimators'
⚠ StackingRegressor failed to instantiate: StackingRegressor.__init__() missing 1 required positional argument: 'estimators'
Evaluating StandardScaler()


100%|██████████| 100/100 [00:00<00:00, 777.05it/s]


❌ StandardScaler: (p_value: 0.000)
⚠ TSNE does not support sample_weight
⚠ TargetEncoder does not support sample_weight
⚠ TfidfTransformer does not support sample_weight
⚠ TheilSenRegressor does not support sample_weight
⚠ TransformedTargetRegressor does not support sample_weight
⚠ TruncatedSVD does not support sample_weight
⚠ TunedThresholdClassifierCV does not support sample_weight
Evaluating TweedieRegressor()


100%|██████████| 100/100 [00:00<00:00, 630.01it/s]

✅ TweedieRegressor: (p_value: 1.000)
⚠ VarianceThreshold does not support sample_weight
⚠ VotingClassifier failed to instantiate: VotingClassifier.__init__() missing 1 required positional argument: 'estimators'
⚠ VotingRegressor failed to instantiate: VotingRegressor.__init__() missing 1 required positional argument: 'estimators'


In [7]:
print(
    f"✅ {len([r for r in deterministic_test_results if r[1] is None])} "
    "passed the deterministic test"
)
print(
    f"❌ {len([r for r in deterministic_test_results if r[1] is not None])} "
    "failed the deterministic test"
)
print(
    f"✅ {len([r for r in statistical_test_results if r.p_value > TEST_THRESHOLD])} "
    "passed the statistical test"
)
print(
    f"❌ {len([r for r in statistical_test_results if r.p_value <= TEST_THRESHOLD])} "
    "failed the statistical test"
)
print(f"❌ {len(errors)} other errors")
print(
    f"⚠ {len(missing_sample_weight_support)} estimators lack sample_weight "
    "support"
)
results_df = pd.DataFrame([r.to_dict() for r in statistical_test_results])

✅ 0 passed the deterministic test
❌ 0 failed the deterministic test
✅ 28 passed the statistical test
❌ 24 failed the statistical test
❌ 10 other errors
⚠ 112 estimators lack sample_weight support


## Details on the statistical test results

In [8]:
results_df.sort_values("p_value")[["estimator_name", "p_value", "deterministic_flag"]]

,estimator_name,p_value,deterministic_flag
46,SGDRegressor,2.208761e-59,False
29,LinearRegression,2.208761e-59,True
26,KernelRidge,2.208761e-59,True
50,StandardScaler,2.208761e-59,True
4,BayesianRidge,2.208761e-59,True
49,SplineTransformer,2.208761e-59,True
48,SVR,2.208761e-59,True
7,CalibratedClassifierCV,2.208761e-59,True
43,RidgeCV,2.208761e-59,True
23,HuberRegressor,2.208761e-59,True


## Details on deterministic test errors

In [8]:
import sys

for est, e in deterministic_test_results:
    if e is None:
        continue

    print(f"❌ {est}: {e}")
    traceback.print_exception(e, file=sys.stdout)
    print()

❌ HuberRegressor(): 
Not equal to tolerance rtol=1e-07, atol=1e-09
Comparing the output of HuberRegressor.predict revealed that fitting with `sample_weight` is not equivalent to fitting with removed or repeated data points.
Mismatched elements: 15 / 15 (100%)
Max absolute difference among violations: 0.00051052
Max relative difference among violations: 7.5912896
 ACTUAL: array([-2.323244e-05,  9.999910e-01,  2.000039e+00,  1.202210e+00,
        2.430282e+00,  9.999892e-01,  1.999998e+00,  2.000024e+00,
        1.656899e+00,  1.999991e+00,  1.576810e+00,  1.295011e+00,
        1.829514e+00,  1.000022e+00,  1.000070e+00])
 DESIRED: array([-2.704185e-06,  9.999971e-01,  2.000005e+00,  1.202141e+00,
        2.430112e+00,  9.999871e-01,  1.999991e+00,  2.000007e+00,
        1.656642e+00,  1.999968e+00,  1.576735e+00,  1.294886e+00,
        1.829381e+00,  1.000010e+00,  9.995597e-01])
Traceback (most recent call last):
  File "/var/folders/_y/lfnx34p13w3_sr2k12bjb05w0000gn/T/ipykernel_47673/

## Details on other errors

In [9]:
import sys

for est, e in errors:
    print(f"❌ {est}: {e}")
    traceback.print_exception(e, file=sys.stdout)
    print()

❌ RANSACRegressor(): Weights sum to zero, can't be normalized
Traceback (most recent call last):
  File "/var/folders/_y/lfnx34p13w3_sr2k12bjb05w0000gn/T/ipykernel_47673/2096092967.py", line 49, in <module>
    result = check_weighted_repeated_estimator_fit_equivalence(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ogrisel/code/sample-weight-audit-nondet/src/sample_weight_audit/estimator_check.py", line 83, in check_weighted_repeated_estimator_fit_equivalence
    predictions_weighted, predictions_repeated, _ = multifit_over_weighted_and_repeated(
                                                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ogrisel/code/sample-weight-audit-nondet/src/sample_weight_audit/estimator_check.py", line 302, in multifit_over_weighted_and_repeated
    est_weighted = check_pipeline_and_fit(
                   ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ogrisel/code/sample-weight-audit-nondet/src/sample_weight_audit/estimator_c

## List of estimators with missing sample_weight support

In [10]:
for est_name in missing_sample_weight_support:
    print(est_name)

ARDRegression
AdditiveChi2Sampler
AffinityPropagation
AgglomerativeClustering
BernoulliRBM
Binarizer
Birch
CCA
ClassifierChain
ColumnTransformer
DictVectorizer
DictionaryLearning
FactorAnalysis
FastICA
FeatureAgglomeration
FeatureHasher
FeatureUnion
FixedThresholdClassifier
FunctionTransformer
GaussianProcessClassifier
GaussianProcessRegressor
GaussianRandomProjection
GenericUnivariateSelect
HDBSCAN
HashingVectorizer
IncrementalPCA
Isomap
KNNImputer
KNeighborsClassifier
KNeighborsRegressor
KNeighborsTransformer
KernelCenterer
KernelPCA
LabelBinarizer
LabelEncoder
LabelPropagation
LabelSpreading
Lars
LarsCV
LassoLars
LassoLarsCV
LassoLarsIC
LatentDirichletAllocation
LinearDiscriminantAnalysis
LocallyLinearEmbedding
MaxAbsScaler
MeanShift
MinMaxScaler
MiniBatchDictionaryLearning
MiniBatchNMF
MiniBatchSparsePCA
MissingIndicator
MultiLabelBinarizer
MultiTaskElasticNet
MultiTaskElasticNetCV
MultiTaskLasso
MultiTaskLassoCV
NMF
NearestCentroid
NeighborhoodComponentsAnalysis
Normalizer
Nystroe